# Time-Series



In [1]:
# Initial imports
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense
from pathlib import Path
import pandas as pd
# Custom classes for running financials and interfacing with SP500 data
from utils.financials import VBot
from utils.trades import VBotTrade

import warnings
warnings.filterwarnings("ignore")

## Read Data

In [2]:
filename = 'sp500'
url='.'
trades = VBotTrade(url, filename)

# Parse into dataframes: <TICKER>_df This reads the entire data file for each ticker. very slow, but selective.
ticker_list = ['AAPL', 'AMZN']

df=pd.DataFrame({'ticker' : ticker_list, 'values':list(ticker_list)})

for ticker in df['ticker'].unique():
    df_name = ticker + '_df'
    globals()[df_name] = trades.get_trades_element([ticker]).drop(columns=["Ticker"])
    globals()[df_name].set_index('Date', inplace=True)
    globals()[df_name]['PctChange'] = globals()[df_name]['Close'].pct_change()
    globals()[df_name]['CumulativeReturn'] = ((1 + globals()[df_name]['PctChange']).cumprod() - 1)
    globals()[df_name] = globals()[df_name].dropna()
    print(f'Created `{df_name}` dataframe.\n')
    # print(globals()[df_name].info())
    display(globals()[df_name].head(3))
    # display(globals()[df_name].tail(3))
    print('___'*30,'\n\n\n\n')

Created `AAPL_df` dataframe.



,High,Low,Open,Close,Volume,Adj Close,PctChange,CumulativeReturn
Date,,,,,,,,
2010-01-05,7.699643,7.616071,7.664286,7.656429,601904800,6.556003,0.001729,0.001729
2010-01-06,7.686786,7.526786,7.656429,7.534643,552160000,6.451720,-0.015906,-0.014205
2010-01-07,7.571429,7.466071,7.562500,7.520714,477131200,6.439795,-0.001849,-0.016027


__________________________________________________________________________________________ 




Created `AMZN_df` dataframe.



,High,Low,Open,Close,Volume,Adj Close,PctChange,CumulativeReturn
Date,,,,,,,,
2010-01-05,135.479996,131.809998,133.429993,134.690002,8851900,134.690002,0.005900,0.005900
2010-01-06,134.729996,131.649994,134.600006,132.250000,7178800,132.250000,-0.018116,-0.012323
2010-01-07,132.320007,128.800003,132.009995,130.000000,11030200,130.000000,-0.017013,-0.029126


__________________________________________________________________________________________ 






## Prepare Data

In [5]:
# Generate the Input Features, X on a new copy of the dataset
from finta import TA

df = AAPL_df.copy()

short_window = 13
long_window = 28

df["SMA_Fast"] = df["Close"].rolling(window=short_window).mean() #TA.SMA(df, 3)
df["SMA_Slow"] = df["Close"].rolling(window=long_window).mean()
df["SSMA"] = TA.SSMA(df)
df["EMA"] = TA.EMA(df, 50)
df["DEMA"] = TA.DEMA(df)
df["TEMA"] = TA.TEMA(df)
df["TRIMA"] = TA.TRIMA(df)

# df['SMA_Fast'].head(50)

In [7]:
from pandas.tseries.offsets import DateOffset
import hvplot.pandas

# Make a plot
start_date = df.index.min()
end_date = start_date + DateOffset(years=6)

mydf = df.copy().dropna()
# mydf = stacked_df.loc[(stacked_df['Ticker'] == ticker)].copy()
# stock_close = mydf[["AAPL_Close"]].loc[start_date:end_date].hvplot(line_color='lightblue', line_width=10, alpha=0.8, ylabel='Price in $', width=1200, height=500)
# moving_avgs = mydf[["AAPL_SMA_Fast", "AAPL_SMA_Slow"]].loc[start_date:end_date].hvplot( ylabel='Price in $', width=1200, height=500)
# sma_ema_plot = stock_close * moving_avgs
# sma_ema_plot

stock_close = mydf[["Close"]].hvplot(line_color="lightblue", line_width=10, alpha=0.8, ylabel="Price in $", width=1200, height=500)
moving_avgs = mydf[["SMA_Fast", "SMA_Slow", "SSMA", "EMA", "DEMA", "TEMA", "TRIMA"]].hvplot( ylabel="Price in $", width=1200, height=500)
sma_ema_plot = stock_close * moving_avgs
sma_ema_plot
# mydf["SMA_Fast"].head()

:Overlay
   .Curve.I     :Curve   [Date]   (Close)
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [Date]   (value)